In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW sale_by_date_employee
AS
SELECT
       DD.Date, DD.CalendarMonthLabel
 , DD.Day, DD.ShortMonth Month, CalendarYear Year
      ,DE.PreferredName, DE.Employee
      ,SUM(FS.TotalExcludingTax) SumOfTotalExcludingTax
      ,SUM(FS.TaxAmount) SumOfTaxAmount
      ,SUM(FS.TotalIncludingTax) SumOfTotalIncludingTax
      ,SUM(Profit) SumOfProfit 
FROM wwilakehouse.fact_sale FS
INNER JOIN wwilakehouse.dimension_date DD ON FS.InvoiceDateKey = DD.Date
INNER JOIN wwilakehouse.dimension_Employee DE ON FS.SalespersonKey = DE.EmployeeKey
GROUP BY DD.Date, DD.CalendarMonthLabel, DD.Day, DD.ShortMonth, DD.CalendarYear, DE.PreferredName, DE.Employee
ORDER BY DD.Date ASC, DE.PreferredName ASC, DE.Employee ASC

In [2]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW cms_provider_dim_drug_lhvw
AS
SELECT Brnd_Name
,Gnrc_Name
,MAX(fourdigityear) AS Max_Year
,MIN(fourdigityear) AS Min_Year
,row_number() OVER (ORDER BY Gnrc_Name ASC) AS drug_key
FROM medicarepartdtest
GROUP BY Brnd_Name,Gnrc_Name

StatementMeta(, a323a35c-6347-408b-8f5c-c1bdb81e8abd, 3, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
cms_provider_dim_drug = spark.sql("SELECT * FROM cms_provider_dim_drug_lhvw")
cms_provider_dim_drug.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/cms_provider_dim_drug_lhvw")

StatementMeta(, a323a35c-6347-408b-8f5c-c1bdb81e8abd, 10, Finished, Available)

In [6]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW cms_dim_year_lhvw
AS
SELECT DISTINCT fourdigityear as Year 
FROM medicarepartdtest

StatementMeta(, a323a35c-6347-408b-8f5c-c1bdb81e8abd, 7, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
cms_dim_year = spark.sql("SELECT * FROM cms_dim_year_lhvw")
cms_dim_year.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/cms_dim_year_lhvw")

StatementMeta(, a323a35c-6347-408b-8f5c-c1bdb81e8abd, 11, Finished, Available)

In [12]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW cms_provider_dim_geography_lhvw
AS
SELECT Prscrbr_City
,Prscrbr_State_Abrvtn
,Prscrbr_State_FIPS
,MAX(fourdigityear) AS Max_Year
,MIN(fourdigityear) AS Min_Year
,row_number() OVER (ORDER BY Prscrbr_State_Abrvtn ASC) AS geo_key
FROM medicarepartdtest
GROUP BY Prscrbr_City,Prscrbr_State_Abrvtn,Prscrbr_State_FIPS

StatementMeta(, a323a35c-6347-408b-8f5c-c1bdb81e8abd, 14, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
cms_provider_dim_geography = spark.sql("SELECT * FROM cms_provider_dim_geography_lhvw")
cms_provider_dim_geography.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/cms_provider_dim_geography_lhvw")

StatementMeta(, a323a35c-6347-408b-8f5c-c1bdb81e8abd, 16, Finished, Available)

In [15]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW cms_provider_dim_provider_lhvw
AS
SELECT Prscrbr_First_Name
,(CASE WHEN Prscrbr_Last_Org_Name IS NULL THEN 'missing' ELSE Prscrbr_Last_Org_Name END) AS Prscrbr_Last_Org_Name
,Prscrbr_NPI
,(CASE WHEN Prscrbr_Type IS NULL THEN 'missing' ELSE Prscrbr_Type END) AS Prscrbr_Type
,Prscrbr_Type_Src
,MAX(fourdigityear) AS Max_Year
,MIN(fourdigityear) AS Min_Year
,row_number() OVER (ORDER BY Prscrbr_NPI ASC) AS provider_key
FROM medicarepartdtest
GROUP BY Prscrbr_First_Name,Prscrbr_Last_Org_Name,Prscrbr_NPI,Prscrbr_Type,Prscrbr_Type_Src

StatementMeta(, a323a35c-6347-408b-8f5c-c1bdb81e8abd, 17, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [16]:
cms_provider_dim_provider = spark.sql("SELECT * FROM cms_provider_dim_provider_lhvw")
cms_provider_dim_provider.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/cms_provider_dim_provider_lhvw")

StatementMeta(, a323a35c-6347-408b-8f5c-c1bdb81e8abd, 18, Finished, Available)

In [2]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW cms_provider_fact_no_null_key_lhvw
AS
SELECT Brnd_Name
,GE65_Bene_Sprsn_Flag
,GE65_Sprsn_Flag
,GE65_Tot_30day_Fills
,GE65_Tot_Benes
,GE65_Tot_Clms
,GE65_Tot_Day_Suply
,GE65_Tot_Drug_Cst
,Gnrc_Name
,Prscrbr_City
,Prscrbr_First_Name
,(CASE WHEN Prscrbr_Last_Org_Name IS NULL THEN 'missing' ELSE Prscrbr_Last_Org_Name END) AS Prscrbr_Last_Org_Name
,Prscrbr_NPI
,Prscrbr_State_Abrvtn
,Prscrbr_State_FIPS
,(CASE WHEN Prscrbr_Type IS NULL THEN 'missing' ELSE Prscrbr_Type END) AS Prscrbr_Type
,Prscrbr_Type_Src
,Tot_30day_Fills
,Tot_Benes
,Tot_Clms
,Tot_Day_Suply
,Tot_Drug_Cst
,fourdigityear as Year
FROM medicarepartdtest

StatementMeta(, ee126462-f7bf-46de-9f3c-7db1baccd35d, 3, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
cms_provider_fact_no_null_key = spark.sql("SELECT * FROM cms_provider_fact_no_null_key_lhvw")
cms_provider_fact_no_null_key.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/cms_provider_fact_no_null_key_lhvw")

StatementMeta(, ee126462-f7bf-46de-9f3c-7db1baccd35d, 5, Finished, Available)

In [5]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW cms_provider_fact_star_lhvw
AS
SELECT GE65_Bene_Sprsn_Flag
,GE65_Sprsn_Flag
,GE65_Tot_30day_Fills
,GE65_Tot_Benes
,GE65_Tot_Clms
,GE65_Tot_Day_Suply
,GE65_Tot_Drug_Cst
,Tot_30day_Fills
,Tot_Benes
,Tot_Clms
,Tot_Day_Suply
,Tot_Drug_Cst
,Year
,b.drug_key
,c.geo_key
,d.provider_key
FROM cms_provider_fact_no_null_key_lhvw a
LEFT OUTER JOIN cms_provider_dim_drug_lhvw b ON a.Brnd_Name = b.Brnd_Name AND a.Gnrc_Name = b.Gnrc_Name
LEFT OUTER JOIN cms_provider_dim_geography_lhvw c ON a.Prscrbr_City = c.Prscrbr_City 
LEFT OUTER JOIN cms_provider_dim_provider_lhvw d ON a.Prscrbr_Last_Org_Name = d.Prscrbr_Last_Org_Name AND a.Prscrbr_NPI = d.Prscrbr_NPI AND a.Prscrbr_Type = d.Prscrbr_Type

StatementMeta(, ee126462-f7bf-46de-9f3c-7db1baccd35d, 7, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
cms_provider_fact_star = spark.sql("SELECT * FROM cms_provider_fact_star_lhvw")
cms_provider_fact_star.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/cms_provider_fact_star_lhvw")

StatementMeta(, ee126462-f7bf-46de-9f3c-7db1baccd35d, 8, Finished, Available)